In [2]:
pip install dpctl

Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.0/35.0 MB 12.0 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 201.2/201.2 MB 28.1 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.6/29.6 MB 109.9 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.3/10.3 MB 149.2 MB/s eta 0:00:00 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.1/45.1 MB 87.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 138.5 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 117.9 MB/s eta 0:00:0000:01
Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install scikit-learn-intelex

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 191.5/191.5 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 121.8 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.6/66.6 MB 62.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.4/13.4 MB 121.4 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 301.8/301.8 kB 26.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.6/38.6 MB 95.5 MB/s eta 0:00:00:00:0100:01
Note: you may need to restart the kernel to use updated packages.


In [6]:
import numpy as np
import dpctl
from sklearnex import patch_sklearn,config_context
patch_sklearn()


Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


In [11]:
pip install openpyxl

Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.9/250.9 kB 10.0 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [12]:
import pandas as pd
from sklearnex.model_selection import train_test_split
from sklearnex.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.preprocessing import MinMaxScaler



data = pd.read_excel('preprocessed_data2.xlsx') 

# Display first few rows
print(data.head())

# Handle missing values 
data.dropna(inplace=True)

# Ensure that 'month_name' exists in your DataFrame before converting
if 'month_name' in data.columns:
    # Convert 'month_name' from string to datetime and extract month number
    data['month'] = pd.to_datetime(data['month_name'], format='%B', errors='coerce').dt.month

    # Check for any NaT values after conversion (indicating conversion issues)
    if data['month'].isnull().any():
        print("Warning: Some month names could not be converted. Check your month_name values.")
else:
    raise ValueError("Column 'month_name' does not exist in the DataFrame.")

# Encode the 'type' feature
if 'type' in data.columns:
    data['type_encoded'] = data['type'].astype('category').cat.codes  # Convert 'type' to numeric codes
else:
    raise ValueError("Column 'type' does not exist in the DataFrame.")

# Features and target variable for Linear Regression (month and type as inputs)
X_lr = data[['month', 'type_encoded']]  # Features: month and type (encoded)
y = data['expense']  # Target variable: expense

# Scale target variable for better performance
scaler = MinMaxScaler()
y_scaled = scaler.fit_transform(y.values.reshape(-1, 1))

# Split the data into training and testing sets for Linear Regression
X_train_lr, X_test_lr, y_train_lr, y_test_lr = train_test_split(X_lr, y_scaled.flatten(), test_size=0.2, random_state=42)

# Train Linear Regression model
lr_model = LinearRegression()
lr_model.fit(X_train_lr, y_train_lr)

# Evaluate Linear Regression model
def evaluate_linear_regression(X_test, y_test):
    predictions = lr_model.predict(X_test)
    predictions_inverse_scaled = scaler.inverse_transform(predictions.reshape(-1, 1)).flatten()

    mae = mean_absolute_error(y_test, predictions)
    mse = mean_squared_error(y_test, predictions)
    r_squared = r2_score(y_test, predictions)

    accuracy_percentage = (1 - (abs(y_test - predictions) / (y.max() - y.min()))).mean() * 100

    return predictions_inverse_scaled, accuracy_percentage

# Function to predict expense using Linear Regression
def predict_expense_lr(month_input, type_input):
    new_data_lr = pd.DataFrame({
        'month': [month_input],
        'type_encoded': [type_input]
    })

    # Use the model to predict expense for new input data
    predicted_expense_scaled = lr_model.predict(new_data_lr)

    # Inverse scale prediction to get original expense value
    predicted_expense_inverse_scaled = scaler.inverse_transform(predicted_expense_scaled.reshape(-1, 1)).flatten()[0]

    # Calculate accuracy percentage based on actual values in training set (this is just an example)
    accuracy_percentage = (1 - abs(predicted_expense_inverse_scaled - y.mean()) / (y.max() - y.min())) * 100

    print(f'\nPredicted Expense using Linear Regression for month {month_input} and type {type_input}: {predicted_expense_inverse_scaled}')
    print(f'Accuracy Percentage: {accuracy_percentage:.2f}%')

    return predicted_expense_inverse_scaled, accuracy_percentage

#  Test with example inputs
predicted_lr_expense, lr_accuracy_percentage = predict_expense_lr(month_input=7, type_input=1)


import joblib

# Save the model
joblib.dump(lr_model, 'linear_regression.pkl')

# Load the model 
lr_model_pickle = joblib.load('linear_regression.pkl')

   gender  age month_name  year             type  expense
0    Male   65       July  2010  processed foods      747
1    Male   40  September  2010      electronics    16804
2  Female   66      March  2010       stationary      292
3    Male   30        May  2010         clothing     2326
4  Female   53        May  2010  processed foods      665

Predicted Expense using Linear Regression for month 7 and type 1: 5502.027654123014
Accuracy Percentage: 88.13%
